In [9]:
import pandas as pd
import eli5
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from scipy.stats import skew
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from pandas.core.common import flatten
from sklearn import linear_model
from sklearn.svm import SVR
from math import e
import ast
from collections import Counter
from scipy.stats import pearsonr

import math
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 10)
pd.options.mode.chained_assignment = None


In [7]:
train = pd.read_csv('/home/scott/Downloads/home-credit-default-risk/application_train.csv')

prev = pd.read_csv('/home/scott/Downloads/home-credit-default-risk/previous_application.csv')




bur = pd.read_csv('/home/scott/Downloads/home-credit-default-risk/bureau.csv')
bbal = pd.read_csv('/home/scott/Downloads/home-credit-default-risk/bureau_balance.csv')

In [3]:
train.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.00,406597.50,24700.50,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,1.00
1,100003,0,Cash loans,F,N,N,0,270000.00,1293502.50,35698.50,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
2,100004,0,Revolving loans,M,Y,Y,0,67500.00,135000.00,6750.00,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
3,100006,0,Cash loans,F,N,Y,0,135000.00,312682.50,29686.50,...,0,0,0,0,nan,nan,nan,nan,nan,nan
4,100007,0,Cash loans,M,N,Y,0,121500.00,513000.00,21865.50,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
5,100008,0,Cash loans,M,N,Y,0,99000.00,490495.50,27517.50,...,0,0,0,0,0.00,0.00,0.00,0.00,1.00,1.00
6,100009,0,Cash loans,F,Y,Y,1,171000.00,1560726.00,41301.00,...,0,0,0,0,0.00,0.00,0.00,1.00,1.00,2.00
7,100010,0,Cash loans,M,Y,Y,0,360000.00,1530000.00,42075.00,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
8,100011,0,Cash loans,F,N,Y,0,112500.00,1019610.00,33826.50,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,1.00
9,100012,0,Revolving loans,M,N,Y,0,135000.00,405000.00,20250.00,...,0,0,0,0,nan,nan,nan,nan,nan,nan


Whoa, 122 features. We will need to do some feature selection, because some models will take way too long to learn, and lots of irrelevant features will harm the model's performance.

In [6]:
print(train.columns.tolist())

['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELE

In [10]:
bur

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.00,-153.00,nan,0,91323.00,0.00,nan,0.00,Consumer credit,-131,nan
1,215354,5714463,Active,currency 1,-208,0,1075.00,nan,nan,0,225000.00,171342.00,nan,0.00,Credit card,-20,nan
2,215354,5714464,Active,currency 1,-203,0,528.00,nan,nan,0,464323.50,nan,nan,0.00,Consumer credit,-16,nan
3,215354,5714465,Active,currency 1,-203,0,nan,nan,nan,0,90000.00,nan,nan,0.00,Credit card,-16,nan
4,215354,5714466,Active,currency 1,-629,0,1197.00,nan,77674.50,0,2700000.00,nan,nan,0.00,Consumer credit,-21,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.00,nan,0.00,0,11250.00,11250.00,0.00,0.00,Microloan,-19,nan
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.00,-2493.00,5476.50,0,38130.84,0.00,0.00,0.00,Consumer credit,-2493,nan
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.00,-970.00,nan,0,15570.00,nan,nan,0.00,Consumer credit,-967,nan
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.00,-1513.00,nan,0,36000.00,0.00,0.00,0.00,Consumer credit,-1508,nan
